In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
from datasets import load_dataset

dataset = load_dataset("zhengxuanzenwu/ms-macro-wellformed_only")
dataset

DatasetDict({
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 153725
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 12467
    })
})

In [12]:
df = dataset["test"].to_pandas()
df.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,"[2,662]","{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0], 'pas...",albany mn population,15177,NUMERIC,"[The population of Albany, Minnesota is 2,662. ]"
1,[The Volcano forecast for Apr 12 is 52 degrees...,"{'is_selected': [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]...","current weather in volcano, ca",114414,DESCRIPTION,[The Volcano forecast for Apr 12 is 52 degrees...
2,[Hippocrates],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",____________________ is considered the father ...,9083,DESCRIPTION,[Hippocrates is considered the father of moder...
3,[120 days from the date of the Note.],"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",how many days is an appraisal good for a fanni...,281439,NUMERIC,[An appraisal is good for 120 days from the da...
4,"[From $26,000 to $39,000 a year]","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",average pharmacy tech salary,40287,NUMERIC,[The average salary for a pharmacy technician ...


In [15]:
test = dataset["test"]
test[0]

{'answers': ['2,662'],
 'passages': {'is_selected': [0, 0, 0, 1, 0, 0, 0, 0],
  'passage_text': ['City of Albany, MN Zip Codes. City of Albany, MN Demographic Information. * Demographic data is based on information taken from the 2000 Census. City of Albany, MN covers 1 Area Code. City of Albany, MN covers 1 Zip Code. 15 Cities within 15 Miles of the City of Albany, MN.',
   'Place of birth for U.S.-born residents: 70% of the 56307 zip code residents lived in the same house 5 years ago. Out of people who lived in different houses, 71% lived in this county. Out of people who lived in different counties, 50% lived in Minnesota. 92% of the 56307 zip code residents lived in the same house 1 year ago.',
   'For the unincorporated community in southeast Minnesota named West Albany, see West Albany, Minnesota. Albany is a city in Stearns County, Minnesota, United States. The population was 2,561 at the 2010 census. It is part of the St. Cloud Metropolitan Statistical Area.',
   'Albany, Minne

In [24]:
test.num_rows

12467

In [35]:
counts = {}
indices = {}
size = 100
for i in range(test.num_rows):
    row = test[i]
    query_type = row["query_type"]
    if query_type in counts:
        counts[query_type] += 1
    else:
        counts[query_type] = 1
    if counts[query_type] == size:
        indices[query_type] = i
counts, indices

({'NUMERIC': 3685,
  'DESCRIPTION': 5487,
  'ENTITY': 1077,
  'PERSON': 868,
  'LOCATION': 1350},
 {'NUMERIC': 179,
  'DESCRIPTION': 218,
  'ENTITY': 2403,
  'LOCATION': 2559,
  'PERSON': 3966})

In [39]:
# create new dataset exluding those idx
baseline = test.select(
    (i for i in range(len(test)) if i <= indices[test[i]["query_type"]])
)

In [40]:
baseline

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 500
})

In [41]:
baseline.to_pandas()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,"[2,662]","{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0], 'pas...",albany mn population,15177,NUMERIC,"[The population of Albany, Minnesota is 2,662. ]"
1,[The Volcano forecast for Apr 12 is 52 degrees...,"{'is_selected': [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]...","current weather in volcano, ca",114414,DESCRIPTION,[The Volcano forecast for Apr 12 is 52 degrees...
2,[Hippocrates],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",____________________ is considered the father ...,9083,DESCRIPTION,[Hippocrates is considered the father of moder...
3,[120 days from the date of the Note.],"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",how many days is an appraisal good for a fanni...,281439,NUMERIC,[An appraisal is good for 120 days from the da...
4,"[From $26,000 to $39,000 a year]","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",average pharmacy tech salary,40287,NUMERIC,[The average salary for a pharmacy technician ...
...,...,...,...,...,...,...
495,"[The Pool Shower, Inc. is a Georgia Domestic P...","{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]...",the pool shower company,518269,PERSON,"[The Pool Shower, Inc. is a Georgia Domestic P..."
496,[Hanson],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",longest tenured american football players,442806,PERSON,[Hanson is the longest tenured American footba...
497,[Mount Able Baptist Church is located at the a...,"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0], '...",mt. view baptist in pendleton sc,460250,PERSON,[Mount Able Baptist Church is located at the a...
498,[Honeysuckle Weeks],"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]...",what actress disappeared for a while,549739,PERSON,[The actress disappeared for a while Honeysuck...


In [42]:
baseline.save_to_disk("../data/datasets/ms_macro/")

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [44]:
from datasets import load_from_disk

new_ds = load_from_disk("../data/datasets/ms_macro/")
new_ds

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 500
})

In [45]:
counts = {}
indices = {}
size = 100
for i in range(new_ds.num_rows):
    row = new_ds[i]
    query_type = row["query_type"]
    if query_type in counts:
        counts[query_type] += 1
    else:
        counts[query_type] = 1
    if counts[query_type] == size:
        indices[query_type] = i
counts, indices

({'NUMERIC': 100,
  'DESCRIPTION': 100,
  'ENTITY': 100,
  'PERSON': 100,
  'LOCATION': 100},
 {'NUMERIC': 179,
  'DESCRIPTION': 215,
  'ENTITY': 443,
  'LOCATION': 461,
  'PERSON': 499})

In [46]:
new_ds.to_pandas()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,"[2,662]","{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0], 'pas...",albany mn population,15177,NUMERIC,"[The population of Albany, Minnesota is 2,662. ]"
1,[The Volcano forecast for Apr 12 is 52 degrees...,"{'is_selected': [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]...","current weather in volcano, ca",114414,DESCRIPTION,[The Volcano forecast for Apr 12 is 52 degrees...
2,[Hippocrates],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",____________________ is considered the father ...,9083,DESCRIPTION,[Hippocrates is considered the father of moder...
3,[120 days from the date of the Note.],"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",how many days is an appraisal good for a fanni...,281439,NUMERIC,[An appraisal is good for 120 days from the da...
4,"[From $26,000 to $39,000 a year]","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",average pharmacy tech salary,40287,NUMERIC,[The average salary for a pharmacy technician ...
...,...,...,...,...,...,...
495,"[The Pool Shower, Inc. is a Georgia Domestic P...","{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]...",the pool shower company,518269,PERSON,"[The Pool Shower, Inc. is a Georgia Domestic P..."
496,[Hanson],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",longest tenured american football players,442806,PERSON,[Hanson is the longest tenured American footba...
497,[Mount Able Baptist Church is located at the a...,"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0], '...",mt. view baptist in pendleton sc,460250,PERSON,[Mount Able Baptist Church is located at the a...
498,[Honeysuckle Weeks],"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]...",what actress disappeared for a while,549739,PERSON,[The actress disappeared for a while Honeysuck...


In [47]:
"""
Official evaluation script for QAConv, modified from SQuAD 2.0.

 * Copyright (c) 2021, salesforce.com, inc.
 * All rights reserved.
 * SPDX-License-Identifier: BSD-3-Clause
 * For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause

"""

import collections
import re
import string


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [49]:
result_all = new_ds.map(
    lambda record, idx: {
        "EM": compute_exact(record["wellFormedAnswers"][0], record["answers"][0]),
        "F1": compute_f1(record["wellFormedAnswers"][0], record["answers"][0]),
    },
    batched=False,
    with_indices=True,
)
result_all

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers', 'EM', 'F1'],
    num_rows: 500
})

In [50]:
result_all.to_pandas()

,answers,passages,query,query_id,query_type,wellFormedAnswers,EM,F1
0,"[2,662]","{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0], 'pas...",albany mn population,15177,NUMERIC,"[The population of Albany, Minnesota is 2,662. ]",0,0.285714
1,[The Volcano forecast for Apr 12 is 52 degrees...,"{'is_selected': [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]...","current weather in volcano, ca",114414,DESCRIPTION,[The Volcano forecast for Apr 12 is 52 degrees...,1,1.000000
2,[Hippocrates],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",____________________ is considered the father ...,9083,DESCRIPTION,[Hippocrates is considered the father of moder...,0,0.250000
3,[120 days from the date of the Note.],"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",how many days is an appraisal good for a fanni...,281439,NUMERIC,[An appraisal is good for 120 days from the da...,0,0.631579
4,"[From $26,000 to $39,000 a year]","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",average pharmacy tech salary,40287,NUMERIC,[The average salary for a pharmacy technician ...,0,0.500000
...,...,...,...,...,...,...,...,...
495,"[The Pool Shower, Inc. is a Georgia Domestic P...","{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]...",the pool shower company,518269,PERSON,"[The Pool Shower, Inc. is a Georgia Domestic P...",1,1.000000
496,[Hanson],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",longest tenured american football players,442806,PERSON,[Hanson is the longest tenured American footba...,0,0.250000
497,[Mount Able Baptist Church is located at the a...,"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0], '...",mt. view baptist in pendleton sc,460250,PERSON,[Mount Able Baptist Church is located at the a...,1,1.000000
498,[Honeysuckle Weeks],"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]...",what actress disappeared for a while,549739,PERSON,[The actress disappeared for a while Honeysuck...,0,0.500000


In [53]:
%pip install -q evaluate rouge_score

Note: you may need to restart the kernel to use updated packages.


In [54]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [56]:
result_all = result_all.map(
    lambda record: bleu.compute(
        predictions=[record["answers"][0]], references=[record["wellFormedAnswers"][0]]
    ),
    batched=False,
)
result_all.to_pandas()

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

,answers,passages,query,query_id,query_type,wellFormedAnswers,EM,F1,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,"[2,662]","{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0], 'pas...",albany mn population,15177,NUMERIC,"[The population of Albany, Minnesota is 2,662. ]",0,0.285714,0.000000,"[1.0, 0.0, 0.0, 0.0]",0.000335,0.111111,1,9
1,[The Volcano forecast for Apr 12 is 52 degrees...,"{'is_selected': [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]...","current weather in volcano, ca",114414,DESCRIPTION,[The Volcano forecast for Apr 12 is 52 degrees...,1,1.000000,1.000000,"[1.0, 1.0, 1.0, 1.0]",1.000000,1.000000,14,14
2,[Hippocrates],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",____________________ is considered the father ...,9083,DESCRIPTION,[Hippocrates is considered the father of moder...,0,0.250000,0.000000,"[1.0, 0.0, 0.0, 0.0]",0.000335,0.111111,1,9
3,[120 days from the date of the Note.],"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",how many days is an appraisal good for a fanni...,281439,NUMERIC,[An appraisal is good for 120 days from the da...,0,0.631579,0.327096,"[1.0, 0.875, 0.8571428571428571, 0.83333333333...",0.367879,0.500000,9,18
4,"[From $26,000 to $39,000 a year]","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",average pharmacy tech salary,40287,NUMERIC,[The average salary for a pharmacy technician ...,0,0.500000,0.193040,"[0.875, 0.7142857142857143, 0.5, 0.4]",0.324652,0.470588,8,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"[The Pool Shower, Inc. is a Georgia Domestic P...","{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]...",the pool shower company,518269,PERSON,"[The Pool Shower, Inc. is a Georgia Domestic P...",1,1.000000,1.000000,"[1.0, 1.0, 1.0, 1.0]",1.000000,1.000000,19,19
496,[Hanson],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",longest tenured american football players,442806,PERSON,[Hanson is the longest tenured American footba...,0,0.250000,0.000000,"[1.0, 0.0, 0.0, 0.0]",0.000335,0.111111,1,9
497,[Mount Able Baptist Church is located at the a...,"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0], '...",mt. view baptist in pendleton sc,460250,PERSON,[Mount Able Baptist Church is located at the a...,1,1.000000,1.000000,"[1.0, 1.0, 1.0, 1.0]",1.000000,1.000000,21,21
498,[Honeysuckle Weeks],"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]...",what actress disappeared for a while,549739,PERSON,[The actress disappeared for a while Honeysuck...,0,0.500000,0.000000,"[1.0, 1.0, 0.0, 0.0]",0.030197,0.222222,2,9


In [57]:
result_all = result_all.map(
    lambda record: rouge.compute(
        predictions=[record["answers"][0]], references=[record["wellFormedAnswers"][0]]
    ),
    batched=False,
)
result_all.to_pandas()

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

,answers,passages,query,query_id,query_type,wellFormedAnswers,EM,F1,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length,rouge1,rouge2,rougeL,rougeLsum
0,"[2,662]","{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0], 'pas...",albany mn population,15177,NUMERIC,"[The population of Albany, Minnesota is 2,662. ]",0,0.285714,0.000000,"[1.0, 0.0, 0.0, 0.0]",0.000335,0.111111,1,9,0.400000,0.250000,0.400000,0.400000
1,[The Volcano forecast for Apr 12 is 52 degrees...,"{'is_selected': [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]...","current weather in volcano, ca",114414,DESCRIPTION,[The Volcano forecast for Apr 12 is 52 degrees...,1,1.000000,1.000000,"[1.0, 1.0, 1.0, 1.0]",1.000000,1.000000,14,14,1.000000,1.000000,1.000000,1.000000
2,[Hippocrates],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",____________________ is considered the father ...,9083,DESCRIPTION,[Hippocrates is considered the father of moder...,0,0.250000,0.000000,"[1.0, 0.0, 0.0, 0.0]",0.000335,0.111111,1,9,0.222222,0.000000,0.222222,0.222222
3,[120 days from the date of the Note.],"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",how many days is an appraisal good for a fanni...,281439,NUMERIC,[An appraisal is good for 120 days from the da...,0,0.631579,0.327096,"[1.0, 0.875, 0.8571428571428571, 0.83333333333...",0.367879,0.500000,9,18,0.640000,0.608696,0.640000,0.640000
4,"[From $26,000 to $39,000 a year]","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",average pharmacy tech salary,40287,NUMERIC,[The average salary for a pharmacy technician ...,0,0.500000,0.193040,"[0.875, 0.7142857142857143, 0.5, 0.4]",0.324652,0.470588,8,17,0.583333,0.454545,0.583333,0.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"[The Pool Shower, Inc. is a Georgia Domestic P...","{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]...",the pool shower company,518269,PERSON,"[The Pool Shower, Inc. is a Georgia Domestic P...",1,1.000000,1.000000,"[1.0, 1.0, 1.0, 1.0]",1.000000,1.000000,19,19,1.000000,1.000000,1.000000,1.000000
496,[Hanson],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",longest tenured american football players,442806,PERSON,[Hanson is the longest tenured American footba...,0,0.250000,0.000000,"[1.0, 0.0, 0.0, 0.0]",0.000335,0.111111,1,9,0.222222,0.000000,0.222222,0.222222
497,[Mount Able Baptist Church is located at the a...,"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0], '...",mt. view baptist in pendleton sc,460250,PERSON,[Mount Able Baptist Church is located at the a...,1,1.000000,1.000000,"[1.0, 1.0, 1.0, 1.0]",1.000000,1.000000,21,21,1.000000,1.000000,1.000000,1.000000
498,[Honeysuckle Weeks],"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]...",what actress disappeared for a while,549739,PERSON,[The actress disappeared for a while Honeysuck...,0,0.500000,0.000000,"[1.0, 1.0, 0.0, 0.0]",0.030197,0.222222,2,9,0.400000,0.250000,0.400000,0.400000
